In [18]:
# You can use python inline. E.g. you can define functions to be used later:

# import environment and expose in jupyter env
# this only works when shellvars is installed as it is in L-CAS Juptyer images
# https://github.com/LCAS/jupyterhub-deploy-docker/blob/master/singleuser/Dockerfile#L100

import shellvars
from os import environ

def loadenv(filename='/opt/ros/kinetic/setup.bash'):
    vs = shellvars.get_vars(filename)
    for v in vs:
        print('%s=%s' % (v.decode("utf-8"), str(vs[v].decode("utf-8"))))
        environ[v.decode("utf-8")] = str(vs[v].decode("utf-8"))

# you can call these functions easily
# (This example is a good way to load environment variables from a config file)

loadenv('/opt/ros/kinetic/setup.bash')
# install marc-hanheide's own python3 version of rosbag_panda: https://github.com/marc-hanheide/RosbagPandas
#   - clone it
#   - run pip install --user -U . in it 
# tutorial at https://nimbus.unl.edu/2014/11/using-rosbag_pandas-to-analyze-rosbag-files/
#sudo apt-get install ros-kinetic-rosbag python-roslz4
def expand_sys_path():
    import sys
    from os import environ

    extra_paths = environ['PYTHONPATH'].split(':')
    sys.path.extend(extra_paths)

    new_path = []
    for p in sys.path:
        if not p in new_path:
            new_path.append(p)

    sys.path = new_path


expand_sys_path()

ROS_ROOT=/opt/ros/kinetic/share/ros
ROS_PACKAGE_PATH=/opt/ros/kinetic/share
ROS_MASTER_URI=http://localhost:11311
ROS_VERSION=1
LD_LIBRARY_PATH=/opt/ros/kinetic/lib:/opt/ros/kinetic/lib/x86_64-linux-gnu
PATH=/opt/ros/kinetic/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
ROS_DISTRO=kinetic
PYTHONPATH=/opt/ros/kinetic/lib/python2.7/dist-packages
PKG_CONFIG_PATH=/opt/ros/kinetic/lib/pkgconfig:/opt/ros/kinetic/lib/x86_64-linux-gnu/pkgconfig
CMAKE_PREFIX_PATH=/opt/ros/kinetic
ROS_ETC_DIR=/opt/ros/kinetic/etc/ros


In [19]:
wheel_radius = .1
robot_radius = 1



In [20]:
# computing the forward kinematics for a differential drive
def forward_kinematics(w_l, w_r):
    c_l = wheel_radius * w_l
    c_r = wheel_radius * w_r
    v = (c_l + c_r) / 2
    a = (c_r - c_l) / (2 * robot_radius)
    return (v, a)



In [21]:
# computing the inverse kinematics for a differential drive
def inverse_kinematics(v, a):
    c_l = v - (robot_radius * a)
    c_r = v + (robot_radius * a)
    w_l = c_l / wheel_radius
    w_r = c_r / wheel_radius
    return (w_l, w_r)

In [27]:
# inverse kinematics from a Twist message (This is what a ROS robot driver has to do)
def inverse_kinematics_from_twist(t):
    return inverse_kinematics(t.linear.x, t.angular.z)

In [28]:
# compute wheel velocities from given linear and angular velocity of the robot
(w_l, w_r) = inverse_kinematics(0.0, 1.0)
print("w_l = %f,\tw_r = %f" % (w_l, w_r))

# compute linear and angular velocities from wheel velocities
(v, a) = forward_kinematics(w_l, w_r)
print ("v = %f,\ta = %f" % (v, a))

# also works with ROS
from geometry_msgs.msg import Twist
t = Twist()

t.linear.x = 0.3
t.angular.z = 0.8

(w_l, w_r) = inverse_kinematics_from_twist(t)
print("w_l = %f,\tw_r = %f" % (w_l, w_r))

w_l = -10.000000,	w_r = 10.000000
v = 0.000000,	a = 1.000000
w_l = -5.000000,	w_r = 11.000000
